In [ ]:
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("../../../shared_data/xlmr_6L")
max_seq_length = 256

In [21]:
def preprocess_data(sample_join_char):
    sample = sample_join_char[0]
    join_char = sample_join_char[1]
    special_char = "▁"
    len_char_original_token = [len(token) for token in sample["words"]]
    new_tokens = tokenizer.tokenize(join_char.join(sample["words"]))
    if len(new_tokens) > max_seq_length * 2:
        return
    new_tags = []
    index_original_token = 0
    next_tag = False
    for token in new_tokens:
        _token = token.replace(special_char, "")
        print([_token, index_original_token, next_tag, len_char_original_token[index_original_token]])
        if index_original_token >= len(len_char_original_token):
            # self.failed_samples["index_oor"].append(sample)
            return
        if len(_token) == len_char_original_token[index_original_token]:
            if next_tag:
                new_tags.append(next_tag)
            else:
                new_tags.append(sample["tags"][index_original_token])
            next_tag = False
            index_original_token += 1
        elif 0 < len(_token) < len_char_original_token[index_original_token]:
            if next_tag:
                new_tags.append(next_tag)
            else:
                new_tags.append(sample["tags"][index_original_token])
                next_tag = (
                    "O"
                    if sample["tags"][index_original_token] == "O"
                    else "I-" + sample["tags"][index_original_token][2:]
                )
            len_char_original_token[index_original_token] -= len(_token)
            if len_char_original_token[index_original_token] == 0:
                index_original_token += 1
                next_tag = False
            elif len_char_original_token[index_original_token] < 0:
                # self.failed_samples["mismatch"].append(sample)
                return
        elif len(_token) > len_char_original_token[index_original_token]:
            # char_len_new_token = len(_token)
            len_merge_token = 0
            merge_tags = set()
            for j in range(index_original_token, len(sample["words"])):
                # print(sample["words"][j], j)
                len_merge_token += len_char_original_token[j]
                merge_tags.add(sample["tags"][j])
                if len_merge_token == len(_token):
                    # print(["merge", merge_tags, _token])
                    if len(merge_tags) == 1:
                        new_tags.append(list(merge_tags)[0])
                        index_original_token = j + 1
                        break
                    else:
                        return
                elif len_merge_token > len(_token):
                    print(len_merge_token, _token)
                    return

        else:
            new_tags.append("X")
    assert len(new_tokens) == len(new_tags), [
        new_tokens,
        new_tags,
        join_char.join(sample["words"]),
    ]
    print(new_tokens, len(new_tokens))
    print(new_tags, len(new_tags))
    # samples.append({"text": join_char.join(sample["words"]), "tags": new_tags, 
    #                     "ori_words": sample["words"], "ori_tags": sample["tags"]})

In [27]:
# sample = {"words": ['ジャパン', 'ジャパンカップサイクルロードレース', 'カップ', 'サイクル', 'ロードレース'], "tags": ['O', 'O', 'O', 'O', 'O']}
# sample = {"words": ["3", "月", "13", "日"], "tags": ["B-PER" ,"O" ,"O" ,"O"]}
# sample = {"words": ["พร้อม", "กัน", "นี้", "พรรค", "มาตุภูมิ", "ได้", "มีการ", "จัด", "ปาฐกถา", "ใน", "หัวข้อ"], 
#           "tags": ["O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O"]}
# sample = {"words": ["아미노", "요시히코", "(", ",", "1928년", "1월", "22일", "~", "2004년", "2월", "27일", ")", "는", "일본", "의", "역사가", "이다", "."],
#           "tags": ["B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-LOC", "O", "O", "O", "O"]}
sample = {
    "words": ['นอกจาก', 'นี้', 'ยัง', 'ขอ', 'ให้', 'รัฐบาล', 'พิจารณา', 'ให้', 'มี', 'รถ', '_', 'ขสมก.', 'ทุก', 'ประเภท', 'ให้', 'บริการ', 'ประชาชน', 'ทุก', 'เส้นทาง', 'บริการ', 'ทั่ว', '_', 'กทม', '.', 'กว่า', '_', '100', '_', 'สาย', '_', 'ให้', 'บริการ', 'ประชาชน', 'อย่าง', 'ทั่วถึง', 'โดย', 'เก็บ', 'ค่า', 'โดยสาร', 'เพียง', 'ครึ่ง', 'ราคา', 'เพื่อ', 'บรรเทา', '_', 'ภาระ', 'ของ', 'ประชาชน', 'ใน', 'ภาวะ', 'เศรษฐกิจ', 'ตกต่ำ'],
    "tags": ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'B-MISC', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
}
preprocess_data((sample, ""))

['아', 0, False, 3]
['미', 0, 'I-PER', 2]
['노', 0, 'I-PER', 1]
['요', 1, False, 4]
['시', 1, 'I-PER', 3]
['히', 1, 'I-PER', 2]
['코', 1, 'I-PER', 1]
['(', 2, False, 1]
[',', 3, False, 1]
['19', 4, False, 5]
['28', 4, 'O', 3]
['년', 4, 'O', 1]
['1', 5, False, 2]
['월', 5, 'O', 1]
['22', 6, False, 3]
['일', 6, 'O', 1]
['~', 7, False, 1]
['2004', 8, False, 5]
['년', 8, 'O', 1]
['2', 9, False, 2]
['월', 9, 'O', 1]
['27', 10, False, 3]
['일', 10, 'O', 1]
[')', 11, False, 1]
['는', 12, False, 1]
['일본', 13, False, 2]
['의', 14, False, 1]
['역', 15, False, 3]
['사가', 15, 'O', 2]
['이다', 16, False, 2]
['.', 17, False, 1]
['▁아', '미', '노', '요', '시', '히', '코', '(', ',', '19', '28', '년', '1', '월', '22', '일', '~', '2004', '년', '2', '월', '27', '일', ')', '는', '일본', '의', '역', '사가', '이다', '.'] 31
['B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O'] 31


In [19]:
tokenizer.tokenize("".join(sample["words"]))

['▁', '中国人民', '解放', '軍', '海', '軍']

In [20]:
tokenizer.encode("".join(sample["words"]))

[0, 6, 107814, 69332, 13668, 2750, 13668, 2]